In [1]:
pip install fastapi uvicorn


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pylint radon flake8


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-multipart


In [6]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastapi import FastAPI, File, UploadFile
import uvicorn
import re
import ast
import asyncio

app = FastAPI()

def analyze_python_code(code: str):
    score = 100
    breakdown = {
        "naming": 10,
        "modularity": 20,
        "comments": 20,
        "formatting": 15,
        "reusability": 15,
        "best_practices": 20
    }
    recommendations = []
    
    # Naming conventions check
    if re.search(r'def [A-Z]', code):
        breakdown["naming"] -= 2
        recommendations.append("Use snake_case for function names in Python.")
    
    # Function length check
    try:
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                if len(node.body) > 10:
                    breakdown["modularity"] -= 5
                    recommendations.append(f"Function '{node.name}' is too long—consider refactoring.")
    except SyntaxError:
        recommendations.append("Code contains syntax errors.")
    
    # Formatting check
    if "\t" in code:
        breakdown["formatting"] -= 3
        recommendations.append("Avoid using tabs, use spaces for indentation.")
    
    # Comments and documentation check
    if not re.search(r'"""|#', code):
        breakdown["comments"] -= 10
        recommendations.append("Add comments or docstrings to improve readability.")
    
    overall_score = sum(breakdown.values())
    return {"overall_score": overall_score, "breakdown": breakdown, "recommendations": recommendations}

@app.post("/analyze-code")
async def analyze_code(file: UploadFile = File(...)):
    content = await file.read()
    code = content.decode("utf-8")
    
    if file.filename.endswith(".py"):
        result = analyze_python_code(code)
    else:
        return {"error": "Unsupported file type"}
    
    return result

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)    


INFO:     Started server process [17176]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
